# Use Stan Model for Bayes Hierarchical RL
- Estimate learning rate (i.e. alpha) for each subject using this model

In [ ]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import os
# import stan
import cmdstanpy
#import pymc3 as pm
#import arviz as az
# import learning package
#import DMpy

In [2]:
# read scr file
scr = pd.read_csv('firstSCRpart.csv')
scr['sub'].unique()

array([ 189,   86,  205,   71,   87,  204,   85,   24,  101,  100,  132,
        185,  167,   43,   62,  105,   73,   82,   30,  160,  196,  146,
       1223,  106,    8,   94,  112,  169, 1222,   55,  170,  131,   47,
        177,   89,  130,  172,  200,  173,   26,   59,    1,   72, 1232,
        166,   32,   11,  104,   16, 1205,  186,   56,   53,  150,   65,
        154,  193,  165,    4,  103,   95,  168,   20,  102,   75,   48,
         27,    5,  136,  182,  202,  114,  121,   81,  203,  108,   80,
        122,   66,   38,  184,  171,  183,   13,   22,  179,  592,  153,
        144,  178,   63,  124,   21,   19,   10,  158,   83,  113,  107,
        126,  152])

In [3]:
# grab just two subjects for now
scrTwo = scr[(scr['sub']==189) | (scr['sub']==152)]
scrTwo.head()

,Event.Nr,CDA.nSCR,CDA.Latency,CDA.AmpSum,CDA.SCR,CDA.ISCR,CDA.PhasicMax,CDA.Tonic,TTP.nSCR,TTP.Latency,TTP.AmpSum,Global.Mean,Global.MaxDeflection,Event.NID,Event.Name,Condition,group,sub
0,1,5,0.8435,0.2852,0.0003,0.1339,8.1296,2.3324,1,3.9335,0.5884,2.4822,0.5884,5,5,CSplusUS1,HC,189
1,2,4,0.7335,0.1033,0.0012,0.4737,0.3046,4.0029,0,NaN,0.0000,4.3933,0.0000,5,5,CSminus1,HC,189
2,3,3,2.9835,0.0783,0.0008,0.3237,0.1352,3.9579,1,2.8335,0.0260,3.9900,0.0154,5,5,CSplus1,HC,189
3,4,1,3.4935,0.1772,0.0002,0.0993,7.0748,3.8756,1,3.5335,0.5186,3.9212,0.5186,5,5,CSplusUS1,HC,189
4,5,0,NaN,0.0000,0.0004,0.1532,0.1604,4.2513,0,NaN,0.0000,4.3461,0.0000,5,5,CSminus1,HC,189


In [4]:
# organize data accordingly
# first, grab just the relevant variables (subject, trial, stimuli, shock)
# we need to generate a new variable shock (1=yes, 0=no)
# we also need a new variable stim (1=CS+, 2=CS-)
scrVec = scrTwo['CDA.AmpSum'].values
print(scrVec.shape)
shockVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture shock (1=yes, 0=no)
stimVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture stimulus (1=CS+, 2= CS-)

for i, cond in enumerate(scrTwo['Condition'].values):
    #print(i)
    if cond=='CSplusUS1':
        shockVec[i]= 1
        stimVec[i] = 1
    else:
        shockVec[i] = 0
        if cond=='CSminus1':
            stimVec[i] = 2
        else:
            stimVec[i] = 1
print(shockVec.shape)
print(stimVec.shape)
nTrials = 30 #len(scrTwo['Event.Nr']) # grab number of trials

(60,)
(60,)
(60,)


In [22]:
Tsubj = [30,30] # set trials per subject
# create matrix for shocks (trials X Subject), scr (trials X subject) and stimulus (trials X subject)
stim = np.reshape(stimVec, (2,30)).T # transform to matrix trials x subject
shock = np.reshape(shockVec, (2,30)).T 
scrMat = np.reshape(scrVec, (2,30)).T
scrVec

array([0.2852, 0.1033, 0.0783, 0.1772, 0.    , 0.    , 0.    , 0.    ,
       0.2063, 0.    , 0.2062, 0.127 , 0.067 , 0.    , 0.011 , 0.    ,
       0.1756, 0.183 , 0.0865, 0.3701, 0.    , 0.    , 0.    , 0.1691,
       0.0112, 0.0439, 0.    , 0.    , 0.    , 0.    , 3.614 , 0.1215,
       0.8997, 1.8746, 0.    , 1.7057, 0.9741, 0.    , 0.9165, 0.0278,
       0.0149, 0.0959, 0.    , 0.    , 0.    , 0.0296, 0.058 , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.0354, 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.0501, 0.    ])

In [23]:
data =  {'N': 2, 'T': 30, 'Tsubj': Tsubj, 'scr': scrMat, 'lambda':shock,
'stim': stim}
rw_bayes = cmdstanpy.CmdStanModel(stan_file='scr_model.stan')


INFO:cmdstanpy:compiling stan program, exe file: /gpfs/ysm/home/oad4/amg_hippo_fc/scr_model
INFO:cmdstanpy:compiler options: stanc_options={}, cpp_options={}
ERROR:cmdstanpy:file /gpfs/ysm/home/oad4/amg_hippo_fc/scr_model.stan, exception Command: ['make', '/gpfs/ysm/home/oad4/amg_hippo_fc/scr_model']
failed, returncode: 2, error: No such file or directory, stderr: In file included from stan/lib/stan_math/lib/eigen_3.3.9/Eigen/src/Core/MatrixBase.h:130:0,
                 from stan/lib/stan_math/lib/eigen_3.3.9/Eigen/Core:441,
                 from stan/lib/stan_math/lib/eigen_3.3.9/Eigen/Dense:1,
                 from stan/lib/stan_math/stan/math/prim/fun/Eigen.hpp:22,
                 from stan/lib/stan_math/stan/math/rev.hpp:4,
                 from stan/lib/stan_math/stan/math.hpp:19,
                 from stan/src/stan/model/model_header.hpp:4,
                 from /gpfs/ysm/home/oad4/amg_hippo_fc/scr_model.hpp:3:
stan/lib/stan_math/stan/math/prim/eigen_plugins.h:18:25: error: exp

ValueError: Unable to compile Stan model file: /gpfs/ysm/home/oad4/amg_hippo_fc/scr_model.stan.

In [ ]:
# Train the model and generate samples
seed = 1332 # set random seed
fit_simple_model = sm.sample(
  data = data,
  chains = 4,
  iter_warmup = 1000,
  iter_sampling = 1000,
  adapt_delta = .9,
  seed = seed
  #inits = 0.2,
)

In [18]:
from cmdstanpy import cmdstan_path, set_cmdstan_path

#oldpath = cmdstan_path()
set_cmdstan_path('/home/oad4/cmdstan')

#newpath = cmdstan_path()

In [16]:
cmdstan_path()

ValueError: no CmdStan binaries found, run command line script "install_cmdstan"